In [ ]:
!git clone https://github.com/pliang279/MultiBench.git

Cloning into 'MultiBench'...
remote: Enumerating objects: 6943, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 6943 (delta 72), reused 121 (delta 60), pack-reused 6789
Receiving objects: 100% (6943/6943), 51.07 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (4258/4258), done.


In [ ]:
%cd MultiBench

/content/MultiBench


In [ ]:
!mkdir data
!pip install gdown && gdown https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU

Downloading...
From (original): https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU
From (redirected): https://drive.google.com/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU&confirm=t&uuid=ba87ac79-8e60-4e3c-930e-2d7a816e2720
To: /content/MultiBench/mosi_raw.pkl
100% 357M/357M [00:01<00:00, 182MB/s]


In [ ]:
import torch
import sys
import os
from torch.utils.data import TensorDataset,DataLoader,ConcatDataset
import torch.nn as nn
from torch.nn.parameter import Parameter

In [ ]:
from datasets.affect.get_data import get_dataloader
traindata, validdata, testdata = get_dataloader(
    '/content/MultiBench/mosi_raw.pkl', robust_test=False, max_pad=True, data_type='mosi', max_seq_len=50)

In [ ]:
for batch in traindata:
  audio,video,text,labels=batch

  break

**Audio**

In [ ]:
all_audio = []
all_labels = []

for batch in traindata:
    audio, labels = batch[0], batch[-1]  # Extract first and last component
    all_audio.append(audio)
    all_labels.append(labels)

all_audio = torch.cat(all_audio, dim=0)
all_labels = torch.cat(all_labels, dim=0)
rounded_labels = torch.round(all_labels)

# Step 2: Clamp the values to the desired range [-2, 2]
clamped_labels = torch.clamp(rounded_labels, min=-2, max=2)

num_classes = 5

# Initialize an empty tensor to store the one-hot encoded labels
one_hot_labels = torch.zeros((len(clamped_labels), num_classes))

# Iterate over each clamped label and set the corresponding index to 1
for i, label in enumerate(clamped_labels):
    # Extract the integer value from the tensor
    int_label = int(label.item())

    # Shift the label range from [-2, 2] to [0, 4]
    shifted_label = int_label + 2
    one_hot_labels[i][shifted_label] = 1
new_dataset = TensorDataset(all_audio, one_hot_labels)
train_audio_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

In [ ]:
print(clamped_labels)

tensor([[-2.],
        [ 2.],
        [ 0.],
        ...,
        [ 2.],
        [-1.],
        [-1.]])


In [ ]:
print(len(train_audio_dataloader))

41


In [ ]:
all_audio = []
all_labels = []

for batch in validdata:
    audio, labels = batch[0], batch[-1]  # Extract first and last component
    all_audio.append(audio)
    all_labels.append(labels)

all_audio = torch.cat(all_audio, dim=0)
all_labels = torch.cat(all_labels, dim=0)
rounded_labels = torch.round(all_labels)

# Step 2: Clamp the values to the desired range [-2, 2]
clamped_labels = torch.clamp(rounded_labels, min=-2, max=2)

num_classes = 5

# Initialize an empty tensor to store the one-hot encoded labels
one_hot_labels = torch.zeros((len(clamped_labels), num_classes))

# Iterate over each clamped label and set the corresponding index to 1
for i, label in enumerate(clamped_labels):
    # Extract the integer value from the tensor
    int_label = int(label.item())

    # Shift the label range from [-2, 2] to [0, 4]
    shifted_label = int_label + 2
    one_hot_labels[i][shifted_label] = 1
new_dataset = TensorDataset(all_audio, one_hot_labels)
val_audio_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

In [ ]:
print(len(val_audio_dataloader))

7


In [ ]:
all_audio = []
all_labels = []

for batch in testdata:
    audio, labels = batch[0], batch[-1]  # Extract first and last component
    all_audio.append(audio)
    all_labels.append(labels)

all_audio = torch.cat(all_audio, dim=0)
all_labels = torch.cat(all_labels, dim=0)
rounded_labels = torch.round(all_labels)

# Step 2: Clamp the values to the desired range [-2, 2]
clamped_labels = torch.clamp(rounded_labels, min=-2, max=2)

num_classes = 5

# Initialize an empty tensor to store the one-hot encoded labels
one_hot_labels = torch.zeros((len(clamped_labels), num_classes))

# Iterate over each clamped label and set the corresponding index to 1
for i, label in enumerate(clamped_labels):
    # Extract the integer value from the tensor
    int_label = int(label.item())

    # Shift the label range from [-2, 2] to [0, 4]
    shifted_label = int_label + 2
    one_hot_labels[i][shifted_label] = 1
new_dataset = TensorDataset(all_audio, one_hot_labels)
test_audio_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

In [ ]:
print(len(test_audio_dataloader))

22


In [ ]:
import torch
import torch.nn as nn

class SubnetModel(nn.Module):
    def __init__(self, input_size, num_utterances, fc1_size, fc2_size, fc3_size):
        super(SubnetModel, self).__init__()

        self.fc1 = nn.Linear(input_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.fc3 = nn.Linear(fc2_size, fc3_size)

        # Final output layer with 5 neurons and sigmoid activation
        self.output_layer = nn.Linear(fc3_size, 5)
        self.sigmoid = nn.Sigmoid()

        self.relu = nn.ReLU()

    def forward(self, x):
        x = torch.mean(x, dim=1)  # Take the mean along the utterance dimension

        # Pass through fully connected layers
        fc1_out = self.relu(self.fc1(x))
        fc2_out = self.relu(self.fc2(fc1_out))
        fc3_out = self.relu(self.fc3(fc2_out))

        # Final output layer with sigmoid activation
        output = self.sigmoid(self.output_layer(fc3_out))

        return output


In [ ]:
max_seq_len = 20
audio_params = (35, max_seq_len, 32, 32, 32)

# Initialize the model
model = SubnetModel(*audio_params)

# Print the model architecture
print(model)


SubnetModel(
  (fc1): Linear(in_features=35, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (output_layer): Linear(in_features=32, out_features=5, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(model.parameters())[2:], lr=0.00006)
num_epochs = 150


In [ ]:
# Training loop
for epoch in range(num_epochs):
    # Set model to training mode
    model.train()

    # Initialize variables for epoch loss and accuracy calculation
    total_loss = 0.0
    correct_predictions_train = 0
    total_predictions_train = 0

    # Iterate over the training dataset
    for inputs, labels in train_audio_dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = loss_fn(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Accumulate total loss
        total_loss += loss.item()

        # Convert outputs to one-hot format
        predicted_labels_train = (outputs >= 0.5).float()

        # Count correct predictions
        correct_predictions_train += (predicted_labels_train == labels).all(dim=1).sum().item()

        # Update total samples
        total_predictions_train += labels.size(0)

    # Calculate average loss for the epoch
    average_loss = total_loss / len(train_audio_dataloader)

    # Calculate training accuracy
    train_accuracy = correct_predictions_train / total_predictions_train

    # Validation
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        correct_predictions_val = 0
        total_predictions_val = 0

        # Iterate over the validation dataset
        for inputs, labels in val_audio_dataloader:
            outputs = model(inputs)
            predicted_labels_val = (outputs >= 0.5).float()
            correct_predictions_val += (predicted_labels_val == labels).all(dim=1).sum().item()
            total_predictions_val += labels.size(0)

        # Calculate validation accuracy
        val_accuracy = correct_predictions_val / total_predictions_val

    # Print epoch information
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')


Epoch [1/150], Training Loss: 1.6077, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [2/150], Training Loss: 1.6075, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [3/150], Training Loss: 1.6070, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [4/150], Training Loss: 1.6066, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [5/150], Training Loss: 1.6066, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [6/150], Training Loss: 1.6066, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [7/150], Training Loss: 1.6063, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [8/150], Training Loss: 1.6060, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [9/150], Training Loss: 1.6059, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [10/150], Training Loss: 1.6050, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [11/150], Training Loss: 1.6051, Training Accuracy: 0.0000, Val

In [ ]:
# Evaluation on test dataset
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct_predictions_test = 0
    total_predictions_test = 0

    # Iterate over the test dataset
    for inputs, labels in test_audio_dataloader:
        outputs = model(inputs)
        predicted_labels_test = (outputs >= 0.5).float()
        correct_predictions_test += (predicted_labels_test == labels).all(dim=1).sum().item()
        total_predictions_test += labels.size(0)

    # Calculate test accuracy
    test_accuracy = correct_predictions_test / total_predictions_test

# Print test accuracy
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.1224


**Video**

In [ ]:
all_video = []
all_labels = []

for batch in traindata:
    video, labels = batch[1], batch[-1]  # Extract first and last component
    all_video.append(video)
    all_labels.append(labels)

all_video = torch.cat(all_video, dim=0)
all_labels = torch.cat(all_labels, dim=0)
rounded_labels = torch.round(all_labels)

# Step 2: Clamp the values to the desired range [-2, 2]
clamped_labels = torch.clamp(rounded_labels, min=-2, max=2)

num_classes = 5

# Initialize an empty tensor to store the one-hot encoded labels
one_hot_labels = torch.zeros((len(clamped_labels), num_classes))

# Iterate over each clamped label and set the corresponding index to 1
for i, label in enumerate(clamped_labels):
    # Extract the integer value from the tensor
    int_label = int(label.item())

    # Shift the label range from [-2, 2] to [0, 4]
    shifted_label = int_label + 2
    one_hot_labels[i][shifted_label] = 1
new_dataset = TensorDataset(all_video, one_hot_labels)
train_video_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

In [ ]:
print(len(train_video_dataloader))

41


In [ ]:
all_video = []
all_labels = []

for batch in validdata:
    video, labels = batch[1], batch[-1]  # Extract first and last component
    all_video.append(video)
    all_labels.append(labels)

all_video = torch.cat(all_video, dim=0)
all_labels = torch.cat(all_labels, dim=0)
rounded_labels = torch.round(all_labels)

# Step 2: Clamp the values to the desired range [-2, 2]
clamped_labels = torch.clamp(rounded_labels, min=-2, max=2)

num_classes = 5

# Initialize an empty tensor to store the one-hot encoded labels
one_hot_labels = torch.zeros((len(clamped_labels), num_classes))

# Iterate over each clamped label and set the corresponding index to 1
for i, label in enumerate(clamped_labels):
    # Extract the integer value from the tensor
    int_label = int(label.item())

    # Shift the label range from [-2, 2] to [0, 4]
    shifted_label = int_label + 2
    one_hot_labels[i][shifted_label] = 1
new_dataset = TensorDataset(all_video, one_hot_labels)
valid_video_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

In [ ]:
print(len(valid_video_dataloader))

7


In [ ]:
all_video = []
all_labels = []

for batch in testdata:
    video, labels = batch[1], batch[-1]  # Extract first and last component
    all_video.append(video)
    all_labels.append(labels)

all_video = torch.cat(all_video, dim=0)
all_labels = torch.cat(all_labels, dim=0)
rounded_labels = torch.round(all_labels)

# Step 2: Clamp the values to the desired range [-2, 2]
clamped_labels = torch.clamp(rounded_labels, min=-2, max=2)

num_classes = 5

# Initialize an empty tensor to store the one-hot encoded labels
one_hot_labels = torch.zeros((len(clamped_labels), num_classes))

# Iterate over each clamped label and set the corresponding index to 1
for i, label in enumerate(clamped_labels):
    # Extract the integer value from the tensor
    int_label = int(label.item())

    # Shift the label range from [-2, 2] to [0, 4]
    shifted_label = int_label + 2
    one_hot_labels[i][shifted_label] = 1
new_dataset = TensorDataset(all_video, one_hot_labels)
test_video_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

In [ ]:
print(len(test_video_dataloader))

22


In [ ]:
max_seq_len = 20
video_params=(35,max_seq_len,32,32,32)

# Initialize the model
model = SubnetModel(*video_params)

# Print the model architecture
print(model)


SubnetModel(
  (fc1): Linear(in_features=35, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (output_layer): Linear(in_features=32, out_features=5, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(model.parameters())[2:], lr=0.00006)
num_epochs = 150


In [ ]:
# Training loop
for epoch in range(num_epochs):
    # Set model to training mode
    model.train()

    # Initialize variables for epoch loss and accuracy calculation
    total_loss = 0.0
    correct_predictions_train = 0
    total_predictions_train = 0

    # Iterate over the training dataset
    for inputs, labels in train_audio_dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = loss_fn(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Accumulate total loss
        total_loss += loss.item()

        # Convert outputs to one-hot format
        predicted_labels_train = (outputs >= 0.5).float()

        # Count correct predictions
        correct_predictions_train += (predicted_labels_train == labels).all(dim=1).sum().item()

        # Update total samples
        total_predictions_train += labels.size(0)

    # Calculate average loss for the epoch
    average_loss = total_loss / len(train_audio_dataloader)

    # Calculate training accuracy
    train_accuracy = correct_predictions_train / total_predictions_train

    # Validation
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        correct_predictions_val = 0
        total_predictions_val = 0

        # Iterate over the validation dataset
        for inputs, labels in val_audio_dataloader:
            outputs = model(inputs)
            predicted_labels_val = (outputs >= 0.5).float()
            correct_predictions_val += (predicted_labels_val == labels).all(dim=1).sum().item()
            total_predictions_val += labels.size(0)

        # Calculate validation accuracy
        val_accuracy = correct_predictions_val / total_predictions_val

    # Print epoch information
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')


Epoch [1/150], Training Loss: 1.6100, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [2/150], Training Loss: 1.6089, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [3/150], Training Loss: 1.6097, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [4/150], Training Loss: 1.6095, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [5/150], Training Loss: 1.6087, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [6/150], Training Loss: 1.6085, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [7/150], Training Loss: 1.6083, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [8/150], Training Loss: 1.6079, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [9/150], Training Loss: 1.6075, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [10/150], Training Loss: 1.6072, Training Accuracy: 0.0000, Validation Accuracy: 0.0000
Epoch [11/150], Training Loss: 1.6071, Training Accuracy: 0.0000, Val

In [ ]:
# Evaluation on test dataset
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct_predictions_test = 0
    total_predictions_test = 0

    # Iterate over the test dataset
    for inputs, labels in test_audio_dataloader:
        outputs = model(inputs)
        predicted_labels_test = (outputs >= 0.5).float()
        correct_predictions_test += (predicted_labels_test == labels).all(dim=1).sum().item()
        total_predictions_test += labels.size(0)

    # Calculate test accuracy
    test_accuracy = correct_predictions_test / total_predictions_test

# Print test accuracy
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.1312
